In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import os

# Set the device to use (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the path to your dataset
dataset_dir = ""

# Set the input size for resizing the images
input_size = (128,128)

In [4]:
# Define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
])

# Load the training dataset
train_dataset = ImageFolder(root=os.path.join(dataset_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load the test dataset
test_dataset = ImageFolder(root=os.path.join(dataset_dir, 'test'), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:

# Define the Bayesian Neural Network model
class BayesianNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=512, dropout_rate=0.5):
        super(BayesianNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Define the training loop
def train(model, optimizer, criterion, train_loader, device):
    model.train()
    train_loss = 0.0
    train_correct = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    train_accuracy = train_correct / len(train_dataset)
    train_loss /= len(train_loader)

    return train_loss, train_accuracy

# Define the evaluation loop
def evaluate(model, criterion, test_loader, device):
    model.eval()
    test_correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = test_correct / len(test_dataset)

    return test_accuracy


In [6]:
# Set the random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Initialize the Bayesian neural network model
input_dim = 3 * input_size[0] * input_size[1]  # Channels * Height *Width of the input images
output_dim = len(train_dataset.classes)  # Number of classes
model = BayesianNet(input_dim, output_dim)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, optimizer, criterion, train_loader, device)
    test_accuracy = evaluate(model, criterion, test_loader, device)

    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f} Train Accuracy: {train_accuracy:.4f} Test Accuracy: {test_accuracy:.4f}")



Epoch [1/10] Train Loss: 2.1643 Train Accuracy: 0.2020 Test Accuracy: 0.3757
Epoch [2/10] Train Loss: 1.8438 Train Accuracy: 0.2778 Test Accuracy: 0.3701
Epoch [3/10] Train Loss: 1.7435 Train Accuracy: 0.3179 Test Accuracy: 0.5591
Epoch [4/10] Train Loss: 1.6181 Train Accuracy: 0.3685 Test Accuracy: 0.5306
Epoch [5/10] Train Loss: 1.5787 Train Accuracy: 0.3911 Test Accuracy: 0.7180
Epoch [6/10] Train Loss: 1.4311 Train Accuracy: 0.4608 Test Accuracy: 0.7003
Epoch [7/10] Train Loss: 1.4737 Train Accuracy: 0.4411 Test Accuracy: 0.6462
Epoch [8/10] Train Loss: 1.3819 Train Accuracy: 0.4768 Test Accuracy: 0.6584
Epoch [9/10] Train Loss: 1.3137 Train Accuracy: 0.5218 Test Accuracy: 0.6073
Epoch [10/10] Train Loss: 1.3203 Train Accuracy: 0.5015 Test Accuracy: 0.6825


In [9]:
# Calculate the final accuracy on the entire test dataset
final_test_accuracy = evaluate(model, criterion, test_loader, device)
print(f"Final Test Accuracy: {final_test_accuracy:.4f}")

Final Test Accuracy: 0.6825
